In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from util import getData
%matplotlib inline

##### ハイパーパラメタの宣言

In [16]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

batch_size = 128
nb_classes = 7
nb_epoch = 12

img_rows, img_cols = 48, 48
nb_filters = 32
pool_size = (2, 2)
kernel_size = (3, 3)

##### データの読み込み

In [5]:
X_train, X_test, y_train, y_test = getData()

In [6]:
X_train.shape

(32633, 2304)

データが既にフラットになっているので、これを4次元データに変換する。(batch size, color, width, height)

In [8]:
## Caution!! ##
# kerasはバックエンドでtheanoもしくはtensorflowを動かしています。
# tensorflowとtheanoでinputとして渡すデータの形式が若干異なります。
# そのため、以下のように、それぞれのフレームワークごとにデータを整形し直します。
if K.image_dim_ordering() == 'th': # theanoがバックエンドで動くように設定されている場合
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:# tensorflowがバックエンドで動くように設定されている場合
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [9]:
X_train.shape

(32633, 48, 48, 1)

4次元になりました。

In [10]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (32633, 48, 48, 1)
32633 train samples
4093 test samples


##### yのone-hot encoding

In [11]:
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.fit_transform(y_test)

##### CNN modelの作成

In [17]:
model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [18]:
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

In [1]:
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, y_test))